In [1]:
#加工後
import xlrd
import pprint
import openpyxl
import numpy as np
from datetime import datetime as dt

start_day = dt(year=2020, month=7, day=30, hour=0)
end_day = dt(year=2021, month=1, day=11, hour=0)
test_start = dt(2020,10,1,0)
#test_end = dt(2021,1,11,0)
day_num = (end_day-start_day).days
total_data_num = day_num+1
test_data_num = (end_day-test_start).days+1
teach_data_num = total_data_num-test_data_num
print(test_data_num)
print(teach_data_num)

data_filename = 'osaka_data_after.xlsx'
save_filename = 'osaka_after100.xlsx'

func_num = 3
x_dimension = 11
total_dimension = func_num**x_dimension
max_iter = 100
l_rate = 0.01
omega_iter = 100

103
63


In [2]:
#データ関連関数
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))

def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

In [3]:
#μやωからyを計算
def cal_y(m, total_m, y, w, omega):
    temp = 0
    for (i, j, l) in zip(m, w, omega):
        temp += i * j * l
    y[1] = temp/total_m

In [4]:
#データを扱いやすく変形
def set_data(y, Y, data, b, c, omega):
    
    for i in range(total_data_num):
        y.append([Y[i][0],0.5])
        
    for i in range(total_dimension):
        omega[i] = (i+1)/total_dimension

In [5]:
#メンバーシップ関数
def fun(c, b, fun_data, data):    
    for i in range(total_data_num):
        for j in range(x_dimension):
            for k in range(func_num):
                #print(i,j,data[i][j])
                if data[i][j] < c[k]-b[k]:
                    fun_data[i][k][j] = 0
                elif data[i][j] > c[k]+b[k]:
                    fun_data[i][k][j] = 0
                elif data[i][j] >= c[k]-b[k] and data[i][j] <= c[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])
                elif data[i][j] > c[k] and data[i][j] <= c[k]+b[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])

In [6]:
#データに関数を適用する関数
def rule(m, total_m, fun_data, data, c, b, w):
    fun(c, b, fun_data, data)
    
    for d_num in range(total_data_num):
        for j in range(x_dimension):
            times = 0
            R = func_num**(x_dimension-1-j)
            for p in range(R):
                for i in range(func_num):
                    r = func_num**j
                    for k in range(r):
                        m[d_num][times] *= fun_data[d_num][i][j]
                        times+=1
                        
    #学習用とテスト用のμの総和
    for total,m1 in zip(total_m,m):
        for m2,W in zip(m1,w):
            total += m2 * W

In [7]:
#最急降下法 時短改良
def gradient_descent(total_m, m, y, w, omega, mse):
    w = np.ones(total_dimension)
    temp_w = 0
    
    #後件部の学習
    for i in range(omega_iter):    
        for Y,Total_m,M in zip(y,total_m,m):
            for Omega,M0,W in zip(omega,M,w):
                if Total_m != 0:
                    Omega -= l_rate * (-Y[0]+Y[1]) * M0 / Total_m
                    temp_w = W - l_rate *  (-Y[0]+Y[1]) * M0 * (Omega-Y[1]) / Total_m
                else:
                    print("total_m is 0")
                '''
                if temp_w < 0:
                    w[dim] = 0
                else:
                    w[dim] = temp_w'''
            temp1 = 0
            temp2 = 0
            for r, W, ome in zip(M, w, omega):
                temp1 += r * W * ome
                temp2 += r * W
            Total_m = temp2
            Y[1] = temp1/Total_m
        MES(y, mse, i, w, omega, 'teach')
        if i%10 == 0:
            print(str(i)+"回目")
    #重みの学習結果を残しておく
    #時間短縮のため
    save_result(y, w, omega)
    
    print("後件部学習終了")
    print("重み学習終了")
    
    #testデータで
    for i in range(teach_data_num,total_data_num):
        temp = 0
        for l,r in zip(m[i],w):
            temp += l*r
        total_m[i] = temp
        
        #エラー処理
        if total_m[i] == 0:
            print(CV_num,i)

        temp = 0
        for r, W, ome in zip(m[i], w, omega):
            temp += r * W * ome
        y[i][1] = temp/total_m[i]

    MES(y, mse, max_iter, w, omega, 'test')
    save_result(y, w, omega)

In [8]:
def MES(y, mse, n, w, omega, litera):
    temp = 0
    
    if litera=='teach':
        for i in range(teach_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse[n] = temp/teach_data_num
    else:
        for i in range(teach_data_num,total_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse = temp/test_data_num
    
        wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
        for sheet_name in wc.sheetnames:
            if sheet_name == 'output':
                wc["sq_error"].cell(column=1, row=1+n, value = mse)

                wc.save('./output/osaka/'+save_filename) 
            else:
                err_sheet = wc.create_sheet('sq_error')
                err_sheet.cell(column=1, row=1+n, value = mse)
    return 

In [9]:
#excelの保存
def save_result(output_list, w, omega):
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            write_list_2d(wc["output"],[['理想値','出力値']], 1, 1)
            write_list_2d(wc["output"],output_list, 2, 1)

            #wc["sq_error"].cell(column=1, row=1, value = mse)
            
            omega_sheet = wc["omega"]
            for i in range(total_dimension):
                omega_sheet.cell(column=1, row=i+1, value = omega[i])
                
            w_sheet = wc["w"]
            for i in range(total_dimension):
                w_sheet.cell(column=1, row=i+1, value = w[i])
            
            wc.save('./output/osaka/'+save_filename)
            print("Already exist. Save complited ")  
            return 
        
    output_sheet = wc.create_sheet('output')
    write_list_2d(output_sheet,[['理想値','出力値']], 1, 1)
    write_list_2d(output_sheet,output_list, 2, 1)
    #err_sheet = wc.create_sheet('sq_error')
    #err_sheet.cell(column=1, row=1, value = mse)
    
    w_sheet = wc.create_sheet('w')
    for i in range(total_dimension):
        w_sheet.cell(column=1, row=i+1, value = w[i])
        
    w_sheet = wc.create_sheet('omega')
    for i in range(total_dimension):
        w_sheet.cell(column=1, row=i+1, value = omega[i])
    
    wc.save('./output/osaka/'+save_filename)
    print("Save complited ")
    return 

In [10]:
#出力yを導出する関数
def make_output(total_m, m, y, fun_data, data, omega, c, b, w, sq_err):
    rule(m, total_m, fun_data, data, c, b, w)
    
    print("学習開始")
    #学習
    gradient_descent(total_m, m, y, w, omega, sq_err)
    
    print("学習完了")
    
    return 0

In [11]:
#データの参照により使えていない
from sklearn import preprocessing

#0~1に変換
def minMax(data, y):
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    y = mm.fit_transform(y)
    pprint.pprint(data)

In [12]:
from sklearn import preprocessing
def main():
    #Excelの指定
    wb = openpyxl.load_workbook('./data/'+data_filename, data_only=True)
    data_sheet = wb['osaka_data_after']

    '''
    data：データ
    y：病床占有率
    fun_data：関数を通したデータの値
    m：ルールに対する適合度
    total_m：適合度の総和
    omega：ルール後件部
    w：ルールの重み
    mse：平均2乗誤差
    c：関数の中心
    b：関数の幅
    '''
    
    start_row = 2
    data = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, 2, x_dimension+1)
    Y = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, x_dimension+2, x_dimension+2)
    y = []
    fun_data = np.empty((total_data_num, func_num, x_dimension))
    m = np.ones((total_data_num, total_dimension))
    total_m = np.zeros(total_data_num)
    omega = np.zeros(total_dimension)
    w = np.ones(total_dimension)
    mse = np.zeros(max_iter)
    
    #中心と幅の初期値の設定
    c = [0, 0.5, 1] 
    b = [1, 0.5, 1] #変えるべき
    
    #データのセット
    set_data(y, Y, data, b, c, omega)
    print("データセット完了")
    
    #参照問題でここで正規化
    #保存して、読み込みでもいいかも
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    y = mm.fit_transform(y)
    print("正規化完了")
    
    make_output(total_m, m, y, fun_data, data, omega, c, b, w, mse)
    
if __name__ == '__main__':
    
    main()
    

データセット完了
正規化完了


KeyboardInterrupt: 